In [5]:
import serial
import time
import re
import statistics

pattern = re.compile(r'([DTH]):\s*(\d+)')

def open_serial(port='COM4', baud=9600, timeout=1):
    ser = serial.Serial(port, baudrate=baud, timeout=timeout)
    ser.reset_input_buffer()
    ser.reset_output_buffer()
    return ser

def request_and_read(ser):
    ser.write(b's')
    ser.flush()
    time.sleep(0.1)
    return ser.readline().decode('ascii', errors='ignore').strip()

def polling_loop(ser, interval):
    print(f"자동 폴링 시작: {interval}초 간격, Ctrl-C로 종료\n")
    dist_list, temp_list, hum_list = [], [], []
    try:
        while True:
            raw = request_and_read(ser)
            m = pattern.match(raw)
            if m:
                kind, val = m.group(1), int(m.group(2))
                if kind == 'D':
                    dist_list.append(val)
                    avg = statistics.mean(dist_list)
                    print(f"[거리] {val} cm  |  누적평균: {avg:.1f} cm (샘플 {len(dist_list)})")
                elif kind == 'T':
                    temp_list.append(val)
                    avg = statistics.mean(temp_list)
                    print(f"[온도] {val} ℃  |  누적평균: {avg:.1f} ℃ (샘플 {len(temp_list)})")
                elif kind == 'H':
                    hum_list.append(val)
                    avg = statistics.mean(hum_list)
                    print(f"[습도] {val} %  |  누적평균: {avg:.1f} % (샘플 {len(hum_list)})")
            else:
                print("응답 형식 오류:", raw)

            time.sleep(interval)
    except KeyboardInterrupt:
        print("\n자동 폴링 중단")

def main():
    ser = open_serial('COM4', baud=9600, timeout=1)
    print("명령어: s → 폴링 시작, q → 종료")
    while True:
        cmd = input("> ").strip().lower()
        if cmd == 'q':
            break
        elif cmd == 's':
            try:
                interval = float(input("폴링 간격(초): "))
            except ValueError:
                interval = 1.0
            polling_loop(ser, interval)
        else:
            print("s 또는 q 만 입력하세요.")
    ser.close()

if __name__ == '__main__':
    main()


명령어: s → 폴링 시작, q → 종료


>  s
폴링 간격(초):  3


자동 폴링 시작: 3.0초 간격, Ctrl-C로 종료

[습도] 60 %  |  누적평균: 60.0 % (샘플 1)
[습도] 58 %  |  누적평균: 59.0 % (샘플 2)
[습도] 60 %  |  누적평균: 59.3 % (샘플 3)
[습도] 64 %  |  누적평균: 60.5 % (샘플 4)
[습도] 68 %  |  누적평균: 62.0 % (샘플 5)
[온도] 29 ℃  |  누적평균: 29.0 ℃ (샘플 1)
[온도] 29 ℃  |  누적평균: 29.0 ℃ (샘플 2)
[온도] 30 ℃  |  누적평균: 29.3 ℃ (샘플 3)
[온도] 30 ℃  |  누적평균: 29.5 ℃ (샘플 4)
[온도] 30 ℃  |  누적평균: 29.6 ℃ (샘플 5)
[온도] 30 ℃  |  누적평균: 29.7 ℃ (샘플 6)
[온도] 30 ℃  |  누적평균: 29.7 ℃ (샘플 7)
[온도] 31 ℃  |  누적평균: 29.9 ℃ (샘플 8)
[온도] 31 ℃  |  누적평균: 30.0 ℃ (샘플 9)
[온도] 31 ℃  |  누적평균: 30.1 ℃ (샘플 10)
[거리] 173 cm  |  누적평균: 173.0 cm (샘플 1)
[거리] 6 cm  |  누적평균: 89.5 cm (샘플 2)
[거리] 12 cm  |  누적평균: 63.7 cm (샘플 3)
[거리] 12 cm  |  누적평균: 50.8 cm (샘플 4)
[거리] 22 cm  |  누적평균: 45.0 cm (샘플 5)
[거리] 173 cm  |  누적평균: 66.3 cm (샘플 6)
[거리] 171 cm  |  누적평균: 81.3 cm (샘플 7)
[거리] 172 cm  |  누적평균: 92.6 cm (샘플 8)

자동 폴링 중단


>  q


In [2]:
import serial
print(serial.__file__)
print(serial.Serial)       # <class 'serial.serialutil.Serial'> 처럼 나와야 합니다.
print(serial.VERSION)

C:\Users\kccistc\anaconda3\envs\myenv\lib\site-packages\serial\__init__.py
<class 'serial.serialwin32.Serial'>
3.5
